# Predicting Links with Graph Neural Networks

In [ ]:
import torch
!pip install -q torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-{torch.__version__}.html

## Graph Autoencoder (VAE) and Variational Graph Autoencoder (VGAE)

In [2]:
import numpy as np
np.random.seed(0)
import torch
import matplotlib.pyplot as plt
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True, split_labels=True, add_negative_train_samples=False),
])

dataset = Planetoid('.', name='Cora', transform=transform)

train_data, val_data, test_data = dataset[0]

In [3]:
from torch_geometric.nn import GCNConv, VGAE

class Encoder(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.conv1 = GCNConv(dim_in, 2 * dim_out)
        self.conv_mu = GCNConv(2 * dim_out, dim_out)
        self.conv_logstd = GCNConv(2 * dim_out, dim_out)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

In [4]:
model = VGAE(Encoder(dataset.num_features, 16)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)
    loss = model.recon_loss(z, train_data.pos_edge_label_index) + (1 / train_data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)

@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    return model.test(z, data.pos_edge_label_index, data.neg_edge_label_index)

for epoch in range(301):
    loss = train()
    val_auc, val_ap = test(test_data)
    if epoch % 50 == 0:
        print(f'Epoch {epoch:>2} | Loss: {loss:.4f} | Val AUC: {val_auc:.4f} | Val AP: {val_ap:.4f}') 

Epoch  0 | Loss: 3.5030 | Val AUC: 0.6891 | Val AP: 0.7247
Epoch 50 | Loss: 1.3045 | Val AUC: 0.6574 | Val AP: 0.7013
Epoch 100 | Loss: 1.1622 | Val AUC: 0.7604 | Val AP: 0.7737
Epoch 150 | Loss: 1.0805 | Val AUC: 0.7811 | Val AP: 0.7875
Epoch 200 | Loss: 0.9752 | Val AUC: 0.8937 | Val AP: 0.8934
Epoch 250 | Loss: 0.9659 | Val AUC: 0.8999 | Val AP: 0.9038
Epoch 300 | Loss: 0.9361 | Val AUC: 0.9038 | Val AP: 0.9085


In [5]:
z = model.encode(test_data.x, test_data.edge_index) 
Ahat = torch.sigmoid(z @ z.T)
Ahat

tensor([[0.8219, 0.3609, 0.6127,  ..., 0.6082, 0.8506, 0.8055],
        [0.3609, 0.9065, 0.8642,  ..., 0.4280, 0.5970, 0.5648],
        [0.6127, 0.8642, 0.8959,  ..., 0.4919, 0.8294, 0.7752],
        ...,
        [0.6082, 0.4280, 0.4919,  ..., 0.6012, 0.6150, 0.5900],
        [0.8506, 0.5970, 0.8294,  ..., 0.6150, 0.9303, 0.8898],
        [0.8055, 0.5648, 0.7752,  ..., 0.5900, 0.8898, 0.8438]],
       device='cuda:0', grad_fn=<SigmoidBackward0>)

In [6]:
z = model.encode(test_data.x, test_data.pos_edge_label_index)
model.decoder(z, test_data.pos_edge_label_index, sigmoid=True)

tensor([0.9165, 0.8298, 0.9254, 0.5981, 0.9876, 0.9603, 0.9712, 0.9983, 0.8475,
        0.9951, 0.9982, 0.7230, 0.9018, 0.8483, 0.9946, 0.9462, 0.9963, 0.9903,
        1.0000, 0.9965, 0.9981, 0.9994, 0.9611, 0.9707, 0.6260, 0.7366, 0.9968,
        0.9239, 0.9674, 0.9983, 0.4993, 0.9964, 0.9910, 0.6425, 0.9964, 0.9894,
        0.8890, 0.6046, 0.6132, 0.9974, 0.9027, 0.9617, 0.9945, 0.9920, 0.9872,
        0.9669, 0.9903, 0.9374, 0.9952, 0.9998, 0.9951, 0.9368, 0.9804, 0.9715,
        0.9159, 0.8235, 0.9775, 0.7406, 0.9910, 0.9983, 0.9349, 0.7455, 0.9966,
        0.9886, 0.9994, 0.5923, 0.7696, 0.8330, 0.6700, 0.8210, 0.9737, 0.9904,
        0.8955, 0.9301, 0.9238, 0.9653, 0.9980, 0.9720, 0.7922, 0.9462, 0.9749,
        0.9525, 0.9997, 0.8246, 0.8182, 0.9983, 0.9984, 0.6248, 0.9507, 0.9751,
        0.7901, 0.9213, 0.6093, 0.9700, 0.9999, 0.9079, 0.9725, 0.8812, 0.9616,
        0.9531, 0.9987, 0.9355, 0.9219, 0.6517, 0.8599, 0.6825, 0.8024, 0.9833,
        0.8606, 0.9638, 0.9090, 0.8912, 

## SEAL

In [7]:
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score
from scipy.sparse.csgraph import shortest_path

import torch.nn.functional as F
from torch.nn import Conv1d, MaxPool1d, Linear, Dropout, BCEWithLogitsLoss

from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, aggr
from torch_geometric.utils import k_hop_subgraph, to_scipy_sparse_matrix

In [8]:
# Load Cora dataset
transform = RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True, split_labels=True)
dataset = Planetoid('.', name='Cora', transform=transform)
train_data, val_data, test_data = dataset[0]
train_data

Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], pos_edge_label=[4488], pos_edge_label_index=[2, 4488], neg_edge_label=[4488], neg_edge_label_index=[2, 4488])

In [9]:
def seal_processing(dataset, edge_label_index, y):
    data_list = []

    for src, dst in edge_label_index.t().tolist():
        sub_nodes, sub_edge_index, mapping, _ = k_hop_subgraph([src, dst], 2, dataset.edge_index, relabel_nodes=True)
        src, dst = mapping.tolist()

        # Remove target link from the subgraph
        mask1 = (sub_edge_index[0] != src) | (sub_edge_index[1] != dst)
        mask2 = (sub_edge_index[0] != dst) | (sub_edge_index[1] != src)
        sub_edge_index = sub_edge_index[:, mask1 & mask2]

        # Double-radius node labeling (DRNL)
        src, dst = (dst, src) if src > dst else (src, dst)
        adj = to_scipy_sparse_matrix(sub_edge_index, num_nodes=sub_nodes.size(0)).tocsr()

        idx = list(range(src)) + list(range(src + 1, adj.shape[0]))
        adj_wo_src = adj[idx, :][:, idx]

        idx = list(range(dst)) + list(range(dst + 1, adj.shape[0]))
        adj_wo_dst = adj[idx, :][:, idx]

        # Calculate the distance between every node and the source target node
        d_src = shortest_path(adj_wo_dst, directed=False, unweighted=True, indices=src)
        d_src = np.insert(d_src, dst, 0, axis=0)
        d_src = torch.from_numpy(d_src)

        # Calculate the distance between every node and the destination target node
        d_dst = shortest_path(adj_wo_src, directed=False, unweighted=True, indices=dst-1)
        d_dst = np.insert(d_dst, src, 0, axis=0)
        d_dst = torch.from_numpy(d_dst)

        # Calculate the label z for each node
        dist = d_src + d_dst
        z = 1 + torch.min(d_src, d_dst) + dist // 2 * (dist // 2 + dist % 2 - 1)
        z[src], z[dst], z[torch.isnan(z)] = 1., 1., 0.
        z = z.to(torch.long)

        # Concatenate node features and one-hot encoded node labels (with a fixed number of classes)
        node_labels = F.one_hot(z, num_classes=200).to(torch.float)
        node_emb = dataset.x[sub_nodes]
        node_x = torch.cat([node_emb, node_labels], dim=1)

        # Create data object
        data = Data(x=node_x, z=z, edge_index=sub_edge_index, y=y)
        data_list.append(data)

    return data_list

In [10]:
# Enclosing subgraphs extraction
train_pos_data_list = seal_processing(train_data, train_data.pos_edge_label_index, 1)
train_neg_data_list = seal_processing(train_data, train_data.neg_edge_label_index, 0)

val_pos_data_list = seal_processing(val_data, val_data.pos_edge_label_index, 1)
val_neg_data_list = seal_processing(val_data, val_data.neg_edge_label_index, 0)

test_pos_data_list = seal_processing(test_data, test_data.pos_edge_label_index, 1)
test_neg_data_list = seal_processing(test_data, test_data.neg_edge_label_index, 0)

In [11]:
train_dataset = train_pos_data_list + train_neg_data_list
val_dataset = val_pos_data_list + val_neg_data_list
test_dataset = test_pos_data_list + test_neg_data_list

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [12]:
class DGCNN(torch.nn.Module):
    def __init__(self, dim_in, k=30):
        super().__init__()

        # GCN layers
        self.gcn1 = GCNConv(dim_in, 32)
        self.gcn2 = GCNConv(32, 32)
        self.gcn3 = GCNConv(32, 32)
        self.gcn4 = GCNConv(32, 1)

        # Global sort pooling
        self.global_pool = aggr.SortAggregation(k=k)

        # Convolutional layers
        self.conv1 = Conv1d(1, 16, 97, 97)
        self.conv2 = Conv1d(16, 32, 5, 1)
        self.maxpool = MaxPool1d(2, 2)

        # Dense layers
        self.linear1 = Linear(352, 128)
        self.dropout = Dropout(0.5)
        self.linear2 = Linear(128, 1)

    def forward(self, x, edge_index, batch):
        # 1. Graph Convolutional Layers
        h1 = self.gcn1(x, edge_index).tanh()
        h2 = self.gcn2(h1, edge_index).tanh()
        h3 = self.gcn3(h2, edge_index).tanh()
        h4 = self.gcn4(h3, edge_index).tanh()
        h = torch.cat([h1, h2, h3, h4], dim=-1)

        # 2. Global sort pooling
        h = self.global_pool(h, batch)

        # 3. Traditional convolutional and dense layers
        h = h.view(h.size(0), 1, h.size(-1))
        h = self.conv1(h).relu()
        h = self.maxpool(h)
        h = self.conv2(h).relu()
        h = h.view(h.size(0), -1)
        h = self.linear1(h).relu()
        h = self.dropout(h)
        h = self.linear2(h).sigmoid()

        return h

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DGCNN(train_dataset[0].num_features).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
criterion = BCEWithLogitsLoss()

def train():
    model.train()
    total_loss = 0

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out.view(-1), data.y.to(torch.float))
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs

    return total_loss / len(train_dataset)

@torch.no_grad()
def test(loader):
    model.eval()
    y_pred, y_true = [], []

    for data in loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        y_pred.append(out.view(-1).cpu())
        y_true.append(data.y.view(-1).cpu().to(torch.float))

    auc = roc_auc_score(torch.cat(y_true), torch.cat(y_pred))
    ap = average_precision_score(torch.cat(y_true), torch.cat(y_pred))

    return auc, ap

for epoch in range(31):
    loss = train()
    val_auc, val_ap = test(val_loader)
    print(f'Epoch {epoch:>2} | Loss: {loss:.4f} | Val AUC: {val_auc:.4f} | Val AP: {val_ap:.4f}')

test_auc, test_ap = test(test_loader)
print(f'Test AUC: {test_auc:.4f} | Test AP {test_ap:.4f}')

Epoch  0 | Loss: 0.6983 | Val AUC: 0.7881 | Val AP: 0.8110
Epoch  1 | Loss: 0.6203 | Val AUC: 0.8498 | Val AP: 0.8785
Epoch  2 | Loss: 0.5868 | Val AUC: 0.8640 | Val AP: 0.8865
Epoch  3 | Loss: 0.5792 | Val AUC: 0.8650 | Val AP: 0.8867
Epoch  4 | Loss: 0.5752 | Val AUC: 0.8733 | Val AP: 0.8896
Epoch  5 | Loss: 0.5725 | Val AUC: 0.8740 | Val AP: 0.8911
Epoch  6 | Loss: 0.5701 | Val AUC: 0.8726 | Val AP: 0.8934
Epoch  7 | Loss: 0.5684 | Val AUC: 0.8739 | Val AP: 0.8937
Epoch  8 | Loss: 0.5675 | Val AUC: 0.8745 | Val AP: 0.8989
Epoch  9 | Loss: 0.5666 | Val AUC: 0.8759 | Val AP: 0.8987
Epoch 10 | Loss: 0.5655 | Val AUC: 0.8651 | Val AP: 0.8890
Epoch 11 | Loss: 0.5642 | Val AUC: 0.8709 | Val AP: 0.8939
Epoch 12 | Loss: 0.5638 | Val AUC: 0.8667 | Val AP: 0.8894
Epoch 13 | Loss: 0.5630 | Val AUC: 0.8674 | Val AP: 0.8926
Epoch 14 | Loss: 0.5617 | Val AUC: 0.8700 | Val AP: 0.8922
Epoch 15 | Loss: 0.5595 | Val AUC: 0.8719 | Val AP: 0.8961
Epoch 16 | Loss: 0.5564 | Val AUC: 0.8695 | Val AP: 0.88